# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [141]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv

# Import API key
from api_keys import geoapify_key

In [142]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,remire-montjoly,4.9167,-52.2667,299.17,89,0,0.00,GF,1709466947
1,1,adamstown,-25.0660,-130.1015,297.68,77,99,7.49,PN,1709466947
2,2,salekhard,66.5300,66.6019,264.41,94,95,3.99,RU,1709466948
3,3,bikenibeu village,1.3673,173.1241,300.13,89,40,5.14,KI,1709466949
4,4,blackmans bay,-43.0167,147.3167,282.22,67,61,1.34,AU,1709466949


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [143]:
coordinates = {
    "City_ID":city_data_df["City_ID"],
    "city":city_data_df["City"],
    "latitude":city_data_df["Lat"],
    "longitude":city_data_df["Lng"],
    "Max Temp":city_data_df["Max Temp"],
    "Humidity":city_data_df["Humidity"],
    "cloudiness":city_data_df["Cloudiness"],
    "Wind Speed":city_data_df["Wind Speed"],
    "Country":city_data_df["Country"],
    "Date":city_data_df["Date"]
    
    
}

coordinates_df = pd.DataFrame(coordinates)

coordinates_df

,City_ID,city,latitude,longitude,Max Temp,Humidity,cloudiness,Wind Speed,Country,Date
0,0,remire-montjoly,4.9167,-52.2667,299.17,89,0,0.00,GF,1709466947
1,1,adamstown,-25.0660,-130.1015,297.68,77,99,7.49,PN,1709466947
2,2,salekhard,66.5300,66.6019,264.41,94,95,3.99,RU,1709466948
3,3,bikenibeu village,1.3673,173.1241,300.13,89,40,5.14,KI,1709466949
4,4,blackmans bay,-43.0167,147.3167,282.22,67,61,1.34,AU,1709466949
...,...,...,...,...,...,...,...,...,...,...
573,573,xinyuan,37.3000,99.0000,274.52,28,28,4.00,CN,1709467324
574,574,rouissat,31.9243,5.3502,296.19,13,0,6.17,DZ,1709467325
575,575,smithers,54.7804,-127.1743,260.33,72,100,4.63,CA,1709467214
576,576,mazagao,-0.1150,-51.2894,299.18,83,100,2.32,BR,1709467326


In [150]:
# Configure the map plot
map_plot_1 = coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=0.01,
    color="city"
)
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (city,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [145]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = coordinates_df[(coordinates_df["Max Temp"]>=293) & (coordinates_df["Max Temp"]<=303) &
(coordinates_df["Humidity"]>=65) & (coordinates_df["Humidity"]<90) &
(coordinates_df["Wind Speed"]<=3)]
                    

# Drop any rows with null values
ideal_weather_df=ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,city,latitude,longitude,Max Temp,Humidity,cloudiness,Wind Speed,Country,Date
0,0,remire-montjoly,4.9167,-52.2667,299.17,89,0,0.00,GF,1709466947
11,11,nueva concepcion,14.2000,-91.3000,297.02,71,27,1.95,GT,1709466953
40,40,divo,5.8397,-5.3600,301.53,75,79,2.40,CI,1709466970
43,43,kone,-21.0595,164.8658,299.07,88,98,0.87,NC,1709466972
72,72,santo antonio,-6.3106,-35.4789,302.65,65,90,1.99,BR,1709466988


### Step 3: Create a new DataFrame called `hotel_df`.

In [146]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["city","Country","latitude","longitude","Humidity"]].copy()
    

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,city,Country,latitude,longitude,Humidity,Hotel Name
0,remire-montjoly,GF,4.9167,-52.2667,89,
11,nueva concepcion,GT,14.2000,-91.3000,71,
40,divo,CI,5.8397,-5.3600,75,
43,kone,NC,-21.0595,164.8658,88,
72,santo antonio,BR,-6.3106,-35.4789,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [147]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel" ,
    "limit":1,
    "apiKey":geoapify_key       
   }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["latitude"]
    lon = row["longitude"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url , params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'city']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


remire-montjoly - nearest hotel: Complexe Belova
nueva concepcion - nearest hotel: No hotel found
divo - nearest hotel: HOTEL AMAWOYE ext.
kone - nearest hotel: Pacifik Appartels
santo antonio - nearest hotel: Pousada Onça Puma
daru - nearest hotel: Kuki
vila velha - nearest hotel: Hotel Prainha
urangan - nearest hotel: Ramada
city of balikpapan - nearest hotel: Hotel Djang Djaya 2
margaret river - nearest hotel: Margaret River Hotel
hithadhoo - nearest hotel: Pebbles Inn
saipan - nearest hotel: Chalan Kanoa Beach Hotel
kingston - nearest hotel: Aphrodite Hotel
peruibe - nearest hotel: Brisamar
rengasdengklok - nearest hotel: No hotel found
madang - nearest hotel: Madang Star International
puerto armuelles - nearest hotel: Coco del Mar
sinabang - nearest hotel: No hotel found
san antonio de pale - nearest hotel: No hotel found
buala - nearest hotel: No hotel found
saldanha - nearest hotel: Hoedjesbaai
bathsheba - nearest hotel: Atlantis Hotel
paramonga - nearest hotel: Hotel Rico
sao r

,city,Country,latitude,longitude,Humidity,Hotel Name
0,remire-montjoly,GF,4.9167,-52.2667,89,Complexe Belova
11,nueva concepcion,GT,14.2000,-91.3000,71,No hotel found
40,divo,CI,5.8397,-5.3600,75,HOTEL AMAWOYE ext.
43,kone,NC,-21.0595,164.8658,88,Pacifik Appartels
72,santo antonio,BR,-6.3106,-35.4789,65,Pousada Onça Puma
105,daru,PG,-9.0763,143.2092,83,Kuki
108,vila velha,BR,-20.3297,-40.2925,69,Hotel Prainha
112,urangan,AU,-25.3000,152.9000,82,Ramada
134,city of balikpapan,ID,-1.2420,116.8942,89,Hotel Djang Djaya 2
136,margaret river,AU,-33.9500,115.0667,89,Margaret River Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
map_plot_1 = coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=0.01,
    color="city"
)

In [170]:
# # Configure the map plot
map_plot2 = hotel_df.hvplot.points(
                                   "longitude",
                                   "latitude",
                                   geo=True,
                                   hover = ["city", "Hotel Name","Country"],
                                   scale = 0.01,
                                    tiles = "OSM",
                                   frame_width=700,
                                   frame_heigh=500,
                                   color="city"
                                   )

# # Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (city)

In [168]:
hotel_coordinates = {
    "city":hotel_df["city"],
    "latitude":hotel_df["latitude"],
    "longitude":hotel_df["longitude"],
    "Humidity":hotel_df["Humidity"],
    "Hotel Name":hotel_df["Hotel Name"],
    "Country":hotel_df["Country"],
    }

hotel_coordinates_df = pd.DataFrame(hotel_coordinates)

map_plot2 = hotel_coordinates_df.hvplot.points(
                                   "longitude",
                                   "latitude",
                                   geo=True,
                                   tiles = "OSM",
                                   hover = ["city","Hotel Name","Country"],                                                                  
                                   frame_width=700,
                                   frame_heigh=500, 
                                   #size="city",
                                   scale = 0.01,                                                                   
                                   color="city"
                                   )
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (city)